In [1]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader

import numpy as np
from PIL import Image

In [2]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 1
no_cuda = False
log_interval = 100

# Model

In [57]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)   # 3 channel, 20개의 filter, 5*5
        self.conv2 = nn.Conv2d(20, 50, 5, 1) # 20 channel, 50개의 filter, 5*5
        self.fc1 = nn.Linear(5*5*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Preprocess

- png형식의 data모두 불러오기

In [58]:
train_paths = glob('dataset/cifar/train/*.png')
test_paths = glob('dataset/cifar/test/*.png')

len(train_paths), len(test_paths)

(50000, 10000)

참고: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class

### customize한 dataset

In [59]:
def get_label(path):
    return os.path.basename(path).split('_')[-1].replace('png','')

label_names = [get_label(path) for path in train_paths]
classes = np.unique(label_names)

In [62]:
def get_onehot(path):
    lbl_name = os.path.basename(path).split('_')[-1].replace('png','')
    onehot = np.argmax(classes==lbl_name)
    return onehot

In [63]:
class Dataset(Dataset):
    def __init__(self, data_paths, transform=None):

        self.data_paths = data_paths
        self.transform = transform

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        path = self.data_paths[idx]
        
        # read image
        image = Image.open(path)#.convert("L")
        
        # get one_hot label
        label = get_onehot(path)
        # label = get_label(path)
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [64]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [65]:
train_loader = torch.utils.data.DataLoader(
    Dataset(train_paths, 
            transforms.Compose([
                transforms.RandomHorizontalFlip(), 
                transforms.ToTensor(), 
                transforms.Normalize(
                    mean=[0.406], 
                    std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=True, 
    **kwargs
)

test_loader = torch.utils.data.DataLoader(
    Dataset(test_paths,
           transforms.Compose([
               transforms.ToTensor(), 
               transforms.Normalize(
                   mean=[0.406], 
                   std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=False, 
    **kwargs
)

In [66]:
next(iter(train_loader))

[tensor([[[[ 0.4962,  0.6356,  0.8971,  ...,  0.4788,  0.3219,  0.0779],
           [ 0.7402,  0.8274,  0.9319,  ...,  0.5136,  0.3393,  0.1476],
           [ 0.8448,  0.8797,  0.8971,  ...,  0.5659,  0.4091,  0.2522],
           ...,
           [ 0.7751,  0.7054,  0.6705,  ...,  0.7228,  0.7925,  0.7751],
           [ 0.5485,  0.5311,  0.5311,  ...,  0.6356,  0.6182,  0.5311],
           [ 0.9668,  0.9145,  0.9494,  ...,  1.0888,  1.1411,  0.8971]],
 
          [[ 0.8971,  1.0017,  1.1062,  ...,  0.8622,  0.6531,  0.4439],
           [ 0.9668,  1.0539,  1.1237,  ...,  0.8797,  0.7054,  0.5136],
           [ 1.0539,  1.1062,  1.1411,  ...,  0.9145,  0.7751,  0.6182],
           ...,
           [ 1.0365,  0.9494,  0.9145,  ...,  1.0017,  1.0191,  0.9842],
           [ 0.8274,  0.8099,  0.8099,  ...,  0.9668,  0.9319,  0.8448],
           [ 1.1585,  1.1062,  1.1062,  ...,  1.3154,  1.3502,  1.1237]],
 
          [[ 0.9668,  1.0191,  1.1237,  ...,  0.4439,  0.2696,  0.1128],
           [ 

In [67]:
for i, data in enumerate(train_loader):
    if i == 0:
        print(data[0].shape, data[1].shape)
        break

torch.Size([64, 3, 32, 32]) torch.Size([64])


# Optimization

In [68]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training

In [ ]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))